In [1]:
import pandas as pd
data=pd.read_csv('creditcard2013.csv')

In [2]:
data.shape

(284807, 31)

In [3]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
data['Amount']=sc.fit_transform(pd.DataFrame(data['Amount']))
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403,0


In [4]:
data.shape

(284807, 31)

In [5]:
data=data.drop(['Time'],axis=1)

In [6]:
data=data.drop_duplicates()

In [7]:
data.shape

(275663, 30)

In [8]:
data['Class'].value_counts()

0    275190
1       473
Name: Class, dtype: int64

In [9]:
x=data.drop('Class',axis=1)
y=data['Class']

In [10]:
from sklearn.model_selection import train_test_split
X_train ,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [11]:
from imblearn.over_sampling import SMOTE
# Initialize SMOTE
smote = SMOTE(random_state=42)

# Fit SMOTE on training data
x_train_resampled, y_train_resampled = smote.fit_resample(X_train, Y_train)

# Check the new class distribution
print(pd.Series(y_train_resampled).value_counts())

0    220148
1    220148
Name: Class, dtype: int64


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# 1. Data Preparation
# Assuming `X` and `y` are the features and labels of your dataset
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# 2. Define Transformer Model
class TransformerPredictionModel(nn.Module):
    def __init__(self, input_dim, n_classes, n_heads=4, n_layers=2, dim_feedforward=128, dropout=0.1):
        super(TransformerPredictionModel, self).__init__()
        self.embedding = nn.Linear(input_dim, dim_feedforward)
        self.pos_encoder = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(dim_feedforward, dim_feedforward)
        )
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(dim_feedforward, n_heads, dim_feedforward, dropout), num_layers=n_layers
        )
        self.fc = nn.Linear(dim_feedforward, n_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x.unsqueeze(1))  # Add sequence dimension
        x = x.mean(dim=1)  # Pooling
        return self.fc(x)

# Initialize model
input_dim = X_train.shape[1]
n_classes = len(torch.unique(y_train))
model = TransformerPredictionModel(input_dim, n_classes)

# 3. Training
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 4. Evaluation
model.eval()
with torch.no_grad():
    predictions = torch.argmax(model(X_test), dim=1)
    print(classification_report(y_test, predictions))


ValueError: could not determine the shape of object type 'Series'

In [9]:
from sklearn.metrics import classification_report

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
log=LogisticRegression()
log.fit(x_train_resampled, y_train_resampled)
y_pred=log.predict(X_test)
print (classification_report(Y_test,y_pred, target_names=['Not Fraud', 'Fraud']))

              precision    recall  f1-score   support

   Not Fraud       1.00      0.98      0.99     55042
       Fraud       0.06      0.88      0.11        91

    accuracy                           0.98     55133
   macro avg       0.53      0.93      0.55     55133
weighted avg       1.00      0.98      0.99     55133



In [11]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Standardize the feature data
scaler = StandardScaler()
x_train_rescaled = scaler.fit_transform(x_train_resampled)
X_test_rescaled = scaler.transform(X_test)

# Initialize the LinearSVC model
linear_svc_model = LinearSVC(C=1.0, max_iter=1000, random_state=42)

# Train the model on the resampled training data
linear_svc_model.fit(x_train_rescaled, y_train_resampled)

# Make predictions on the test data
y_pred = linear_svc_model.predict(X_test_rescaled)

# Print the classification report
print("Classification Report:")
print(classification_report(Y_test, y_pred, target_names=['Not Fraud', 'Fraud']))
 

Classification Report:
              precision    recall  f1-score   support

   Not Fraud       1.00      0.98      0.99     55042
       Fraud       0.08      0.87      0.14        91

    accuracy                           0.98     55133
   macro avg       0.54      0.93      0.57     55133
weighted avg       1.00      0.98      0.99     55133



In [12]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train_resampled ,y_train_resampled)

# Make predictions
y_pred = knn.predict(X_test)
print (classification_report(Y_test,y_pred, target_names=['Not Fraud', 'Fraud']))

              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00     55042
       Fraud       0.53      0.81      0.64        91

    accuracy                           1.00     55133
   macro avg       0.77      0.91      0.82     55133
weighted avg       1.00      1.00      1.00     55133



In [13]:
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
dt = DecisionTreeClassifier(criterion='entropy', max_depth=50, random_state=42)
dt.fit(x_train_resampled ,y_train_resampled)
y_pred = dt.predict(X_test)
print (classification_report(Y_test,y_pred, target_names=['Not Fraud', 'Fraud']))

              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00     55042
       Fraud       0.45      0.77      0.57        91

    accuracy                           1.00     55133
   macro avg       0.72      0.88      0.78     55133
weighted avg       1.00      1.00      1.00     55133



In [14]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, max_depth=50, random_state=42)
rf_model.fit(x_train_resampled ,y_train_resampled)
y_pred = rf_model.predict(X_test)
print (classification_report(Y_test,y_pred, target_names=['Not Fraud', 'Fraud']))

              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00     55042
       Fraud       0.84      0.79      0.81        91

    accuracy                           1.00     55133
   macro avg       0.92      0.90      0.91     55133
weighted avg       1.00      1.00      1.00     55133



In [19]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(n_estimators=100, max_depth=50, learning_rate=0.1, random_state=42)
xgb_model.fit(x_train_resampled ,y_train_resampled)

# Make predictions
y_pred = xgb_model.predict(X_test)
print (classification_report(Y_test,y_pred, target_names=['Not Fraud', 'Fraud']))

              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00     55042
       Fraud       0.75      0.82      0.79        91

    accuracy                           1.00     55133
   macro avg       0.87      0.91      0.89     55133
weighted avg       1.00      1.00      1.00     55133



In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

 

# Assuming you have already performed GAN oversampling and have X_resampled and y_resampled
# Build the Multi-Layer Perceptron (MLP) model
def build_mlp(input_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))  # First hidden layer
    model.add(Dropout(0.2))  # Dropout for regularization
    model.add(Dense(64, activation='relu'))  # Second hidden layer
    model.add(Dropout(0.2))  # Dropout for regularization
    model.add(Dense(32, activation='relu'))  # Third hidden layer
    model.add(Dense(1, activation='sigmoid'))  # Output layer (sigmoid for binary classification)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build and train the MLP model using resampled data
mlp_model = build_mlp(x_train_resampled.shape[1])

# Fit the model on resampled data (X_resampled and y_resampled)
mlp_model.fit(x_train_resampled,y_train_resampled, epochs=50, batch_size=64, validation_split=0.2, verbose=1)

# Make predictions on the test set
y_pred = (mlp_model.predict(X_test) > 0.5).astype(int)  # Converting probabilities to class labels

# Print classification report
print(classification_report(Y_test, y_pred, target_names=['Not Fraud', 'Fraud']))


Epoch 1/50
5504/5504 ━━━━━━━━━━━━━━━━━━━━ 5s 793us/step - accuracy: 0.9797 - loss: 0.0574 - val_accuracy: 0.9997 - val_loss: 0.0017
Epoch 2/50
5504/5504 ━━━━━━━━━━━━━━━━━━━━ 4s 796us/step - accuracy: 0.9979 - loss: 0.0082 - val_accuracy: 1.0000 - val_loss: 0.0034
Epoch 3/50
5504/5504 ━━━━━━━━━━━━━━━━━━━━ 4s 804us/step - accuracy: 0.9985 - loss: 0.0057 - val_accuracy: 0.9998 - val_loss: 0.0024
Epoch 4/50
5504/5504 ━━━━━━━━━━━━━━━━━━━━ 4s 779us/step - accuracy: 0.9988 - loss: 0.0049 - val_accuracy: 0.9995 - val_loss: 0.0032
Epoch 5/50
5504/5504 ━━━━━━━━━━━━━━━━━━━━ 5s 836us/step - accuracy: 0.9989 - loss: 0.0045 - val_accuracy: 0.9998 - val_loss: 9.3610e-04
Epoch 6/50
5504/5504 ━━━━━━━━━━━━━━━━━━━━ 5s 889us/step - accuracy: 0.9990 - loss: 0.0041 - val_accuracy: 1.0000 - val_loss: 4.3657e-04
Epoch 7/50
5504/5504 ━━━━━━━━━━━━━━━━━━━━ 4s 778us/step - accuracy: 0.9992 - loss: 0.0036 - val_accuracy: 1.0000 - val_loss: 4.4926e-04
Epoch 8/50
5504/5504 ━━━━━━━━━━━━━━━━━━━━ 4s 774us/step - accura